In [1]:
import numpy as np
import pandas as pd
from pyDOE import lhs  # Latin Hypercube Sampling
from scipy.optimize import newton
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt
import os
from sklearn.model_selection import GroupShuffleSplit
import pickle

# Ensure reproducibility
np.random.seed(42)
tf.random.set_random_seed(42)


# Material constant ranges from Table 1
material_constant_ranges = {
    "E": [60000e6, 210000e6],  # Young's modulus in MPa
    "sigma_y": [90e6, 1000e6],  # Yield stress in MPa
    "h": [5, 15],  # Isotropic hardening rate
}
# Generate material constants using Latin Hypercube Sampling
def generate_material_constants_lhs(num_samples, ranges):
    num_variables = len(ranges)
    lhs_samples = lhs(num_variables, samples=num_samples)
    material_constants = []

    for i, (key, (low, high)) in enumerate(ranges.items()):
        samples = lhs_samples[:, i] * (high - low) + low
        material_constants.append(samples)

    return np.column_stack(material_constants)


# Generate strain history
def generate_strain_history(num_samples, lb, ub):
    strain_history = np.zeros(num_samples)
    cumulative_strain = 0  # Start with zero cumulative strain

    # First range: Accumulate small positive strain increments
    for i in range(300):
        increment = np.random.uniform(lb, ub)
        cumulative_strain += increment
        strain_history[i] = cumulative_strain

    # Second range: Accumulate negative strain increments
    for i in range(300, 900):
        increment = np.random.uniform(lb, ub)
        cumulative_strain -= increment
        strain_history[i] = cumulative_strain

    # Third range: Return to positive strain increments
    for i in range(900, num_samples):
        increment = np.random.uniform(lb, ub)
        cumulative_strain += increment
        strain_history[i] = cumulative_strain

    return strain_history


# Hooke's law (elastic predictor)
def elastic_predictor(eps, eps_p, E, h, alpha, sigma_y):
    sigma_trial = E * (eps - eps_p)  # Trial stress
    yield_function = np.abs(sigma_trial) - (sigma_y + h * alpha)
    return sigma_trial, yield_function


# Von Mises return mapping algorithm
def von_mises(eps, eps_p, sigma_trial, yield_function, alpha, h, E, sigma_y):
    if yield_function <= 0:
        # Elastic step
        sigma_updated = sigma_trial
        eps_p = eps_p
        alpha = alpha
        delta_gamma=0
    else:
        # Plastic step
        delta_gamma = yield_function / (E + h)
        eps_p = eps_p + delta_gamma* (sigma_trial/np.abs(sigma_trial))
        alpha = alpha + delta_gamma
        sigma_updated = sigma_trial - E * delta_gamma
        sigma_updated = E*(eps-eps_p)

    return sigma_updated, eps_p, alpha, delta_gamma


# Generate dataset
def generate_dataset(num_samples,num_strain_samples,lb, ub,ranges):
    
    dataset = []
    material_constants = generate_material_constants_lhs(num_samples, ranges)
    #print(material_constants)
    
    for constants in material_constants:
        
        E, sigma_y, h = constants
        strain_history = generate_strain_history(num_strain_samples, lb, ub)

        # Initialize state variables
        alpha, eps_p = 0, 0
    
        for eps in strain_history:
            # Elastic predictor
            sigma_trial, yield_function = elastic_predictor(eps, eps_p, E, h, alpha, sigma_y)

            # Return mapping algorithm
            sigma_updated, eps_p, alpha, delta_gamma  = von_mises(eps, eps_p, sigma_trial, yield_function, alpha, h, E, sigma_y)

            # Store the data
            dataset.append([
                E,sigma_y,h, yield_function, eps, sigma_updated, eps_p, alpha, delta_gamma
            ])

    return np.array(dataset)


# Save the dataset to a CSV file
def save_dataset_to_csv(dataset, filename="generated_dataset_von_mises_training_13_12.csv"):
    columns = ["E","sigma_y","h", "yield_function", "strain", "sigma_updated", "cumulative_plastic_strain", "alpha", 'plastic_strain']
    df = pd.DataFrame(dataset, columns=columns)
    df.to_csv(filename, index=False)
    print(f"Dataset saved to {filename}")


In [2]:
def normalize_dataset(data, scaler=None):
    if scaler is None:
        scaler = MinMaxScaler()
        normalized_data = scaler.fit_transform(data)
    else:
        normalized_data = scaler.transform(data)
    return normalized_data, scaler

In [3]:
# Build the neural network
def build_model(input_dim):
    model = Sequential([
        Dense(10, activation='sigmoid', input_shape=(input_dim,)),  # Use 10 neurons, sigmoid activation
        Dense(1, activation='linear')     # Linear activation for the output
    ])
    custom_adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(optimizer=custom_adam, loss='mean_squared_error', metrics=['mae'])
    return model

In [4]:
# Plot training and validation loss
def plot_loss(history, filename='loss_curve.png'):
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss (MSE)')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    #plt.savefig(filename)  # Save the plot as an image file
    #plt.close()  # Close the plot to avoid displaying it in the notebook
    #print(f"Loss curve saved as {filename}")

# Plot predicted vs original values
def plot_predicted_vs_actual(y_test, y_pred, filename='predicted_vs_actual.png'):
    plt.figure(figsize=(8, 6))
    plt.plot(y_test, label='Original Values', color='blue')
    plt.plot(y_pred, label='Predicted Values', color='red', linestyle='--')
    plt.xlabel('Sample index')
    plt.ylabel('Plastic Strain')
    plt.title('Predicted vs. Original Plastic Strain')
    plt.legend()
    plt.grid(True)
    plt.show()
    plt.savefig('predicted_vs_actual')  # Save the plot as an image file
    plt.close()  # Close the plot to avoid displaying it in the notebookLHSLHS
    print(f"Plot saved as {'predicted_vs_actual'}")

In [5]:
training = generate_dataset(num_samples=500, num_strain_samples=1500, lb=0.0001,ub=0.0002, ranges=material_constant_ranges)
#test1 = generate_dataset(lb=0,ub=0.0001, num_strain_samples=1500)
#test2 = generate_dataset(lb=0.0002,ub=0.0004, num_strain_samples=1500)

In [6]:
columns = ["E","sigma_y","h", "yield_function", "strain", "sigma_updated", "cumulative_plastic_strain", "alpha", 'plastic_strain']
df = pd.DataFrame(training, columns=columns)

In [7]:
df.tail(50)

,E,sigma_y,h,yield_function,strain,sigma_updated,cumulative_plastic_strain,alpha,plastic_strain
749950,1.510139e+11,2.951207e+08,5.91406,1.664411e+07,0.035163,2.951207e+08,0.033209,0.205843,0.000110
749951,1.510139e+11,2.951207e+08,5.91406,2.811409e+07,0.035349,2.951207e+08,0.033395,0.206029,0.000186
749952,1.510139e+11,2.951207e+08,5.91406,2.342001e+07,0.035504,2.951207e+08,0.033550,0.206184,0.000155
749953,1.510139e+11,2.951207e+08,5.91406,1.798211e+07,0.035623,2.951207e+08,0.033669,0.206303,0.000119
749954,1.510139e+11,2.951207e+08,5.91406,1.759746e+07,0.035740,2.951207e+08,0.033786,0.206420,0.000117
749955,1.510139e+11,2.951207e+08,5.91406,2.079861e+07,0.035878,2.951207e+08,0.033923,0.206557,0.000138
749956,1.510139e+11,2.951207e+08,5.91406,2.565076e+07,0.036048,2.951207e+08,0.034093,0.206727,0.000170
749957,1.510139e+11,2.951207e+08,5.91406,2.911013e+07,0.036240,2.951207e+08,0.034286,0.206920,0.000193
749958,1.510139e+11,2.951207e+08,5.91406,2.369288e+07,0.036397,2.951207e+08,0.034443,0.207077,0.000157
749959,1.510139e+11,2.951207e+08,5.91406,2.997249e+07,0.036596,2.951207e+08,0.034641,0.207275,0.000198


In [8]:
save_dataset_to_csv(training)

Dataset saved to generated_dataset_von_mises_training_13_12.csv


In [9]:
df.shape

(750000, 9)

In [10]:
# Separate features and target
X = training[:, :4]  # Features (E, sigma_y, c1, c2, c3, gamma1, gamma2, gamma3, b, Q, strain)
y = training[:, -1]   # Targets (plastic_strain)

In [12]:
#X_normalized, scaler_X= normalize_dataset(X)
#y = y.reshape(-1, 1)
#y_normalized, scaler_y= normalize_dataset(y)

In [13]:
# Build the neural network model
print("Building the neural network model...")
model = build_model(X.shape[1])

# Define the early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',         # Monitor the validation loss
    patience=10,                # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True   # Restore model weights from the epoch with the best value of the monitored quantity
)

# Train the model with early stopping
print("Training the model...")
history = model.fit(
    X, 
    y, 
    epochs=2000,               # Set a large number of epochs
    batch_size=1000,
    validation_split=0.2, 
    callbacks=[early_stopping],  # Include the early stopping callback
    verbose=1
)

Building the neural network model...
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training the model...
Train on 600000 samples, validate on 150000 samples
Epoch 1/2000
600000/600000 [==============================] - 1s 1us/sample - loss: 0.0628 - mean_absolute_error: 0.1507 - val_loss: 3.6673e-08 - val_mean_absolute_error: 1.8476e-04
Epoch 2/2000
600000/600000 [==============================] - 0s 1us/sample - loss: 4.1126e-09 - mean_absolute_error: 4.5095e-05 - val_loss: 2.5454e-09 - val_mean_absolute_error: 3.6750e-05
Epoch 3/2000
600000/600000 [==============================] - 0s 1us/sample - loss: 2.8598e-09 - mean_absolute_error: 3.8718e-05 - val_loss: 2.5461e-09 - val_mean_absolute_error: 3.6775e-05
Epoch 4/2000
600000/600000 [==============================] - 0s 1us/sample - loss: 2.8599e-09 - mean_absolute_error: 3.8721e-05 - val_loss: 2.5441e-09 - val_mean_absolute_error: 3.6703e-05
Epoch 5/2000
600000

In [14]:
# Save the model
model.save('plastic_strain_predictor_model_von_mises_18_12_Unnormalized.h5')
#print("Model saved as 'plastic_strain_predictor_model.h5'")

In [15]:
training[:, :4]

array([[ 1.48205913e+11,  2.55274406e+08,  1.09658630e+01,
        -2.30266046e+08],
       [ 1.48205913e+11,  2.55274406e+08,  1.09658630e+01,
        -2.15082373e+08],
       [ 1.48205913e+11,  2.55274406e+08,  1.09658630e+01,
        -1.95724646e+08],
       ...,
       [ 1.51013908e+11,  2.95120657e+08,  5.91406038e+00,
         1.92857538e+07],
       [ 1.51013908e+11,  2.95120657e+08,  5.91406038e+00,
         2.44384065e+07],
       [ 1.51013908e+11,  2.95120657e+08,  5.91406038e+00,
         1.95273100e+07]])